In [1]:
''' 

Submitter : JESOON KANG, 20170937
Date : 2019. 10. 


    Assignment 4. 

-   -


'''
import matplotlib.pyplot as plt
import math
import numpy as np
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

import torchvision
import os




In [2]:
# predict Function. if h(x) returns >=0.5, set to 1. other cases, set to 0.
def predict(h, labels) :
    mount = len(h)
    correct = 0
    
    for i in range(0,mount) :
        if h[i] >= 0.5 :
            if labels[i] == 1 :
                correct +=1
        else :
            if labels[i] == 0 :
                correct +=1
    return correct * (1/mount)

#### Section 2 END ####

In [270]:

#### Section 1. #### This Section is bringed Data_import_ex.py file.

# Image Data import & resize

transform = transforms.Compose([#transforms.Resize((256,256)),  
                                transforms.Grayscale(),		# the code transforms.Graysclae() is for changing the size [3,100,100] to [1, 100, 100] (notice : [channel, height, width] )
                                transforms.ToTensor(),])

#train_data_path = 'relative path of training data set'
train_data_path = '../data/horse-or-human/train'
trainset = torchvision.datasets.ImageFolder(root=train_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
# if shuffle=True, the data reshuffled at every epoch 
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=True, num_workers=0)  


validation_data_path = '../data/horse-or-human/validation'
valset = torchvision.datasets.ImageFolder(root=validation_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
valloader = torch.utils.data.DataLoader(valset, batch_size=1, shuffle=False, num_workers=0)  

#### Section 1 END ####


In [276]:
#### Section 3 START ####
# Section 3 includes Data Pre-processing. ReDesign datasets to easy calculate

# Data reconstruct. vectorize

# Each Image file will be stored shape of a row
training_vectorized = []
training_labels = []


tmp = 0
# Training data vectorizing
for i, data in enumerate(trainloader) :
    train_data = []
    inputs, labels = data
#    print(tmp)
    tmp += 1
    for u in inputs :
        for col in u[0] :
            train_data += list(col)
    training_vectorized.append(train_data)
    training_labels.append([labels])

training_vectorized = torch.Tensor(training_vectorized)
training_labels = torch.Tensor(training_labels)
print("train set is loaded")



train set is loaded


In [277]:
print(training_vectorized)

tensor([[0.1216, 0.1137, 0.2235,  ..., 0.6157, 0.6196, 0.5882],
        [0.7647, 0.7608, 0.7608,  ..., 0.7569, 0.7647, 0.7686],
        [0.2549, 0.3333, 0.3725,  ..., 0.5373, 0.5020, 0.5098],
        ...,
        [0.7412, 0.7373, 0.7373,  ..., 0.7137, 0.7137, 0.6941],
        [0.7569, 0.7608, 0.7647,  ..., 0.3529, 0.4510, 0.4784],
        [0.7412, 0.7647, 0.8157,  ..., 0.1529, 0.1882, 0.1882]])


In [280]:

validation_vectorized = []
validation_labels = []

# Validation data vectorizing
for i, data in enumerate(valloader) :
    val_data = []
    inputs, labels = data
    for u in inputs :
        for col in u[0] :
            val_data += list(col)
    validation_vectorized.append(val_data)
    validation_labels.append([labels])
validation_vectorized = torch.Tensor(validation_vectorized)
validation_labels = torch.Tensor(validation_labels)

#### Section 3 END ####



In [281]:
print(validation_vectorized)

tensor([[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        ...,
        [1.0000, 1.0000, 1.0000,  ..., 0.9961, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 0.9961, 0.9961, 0.9961],
        [1.0000, 1.0000, 1.0000,  ..., 0.9961, 0.9961, 0.9961]])


In [282]:

# sigmoid Function.
def sigmoid(z) :
    return 1 / (1 + torch.pow(math.e,-z/100))

def get_activation(z,type) :
    if (type == 0) :
        return sigmoid(z)
    else :
        print("Error, get_activation")
        return 0

def get_derv_activation(z,type) :
    if (type == 0) :
        return sigmoid(z)*(1-sigmoid(z))
    else :
        print("Error, get_derv_activation")
        return 0                                                                  

In [306]:

# log variables setting : to record statements
log_training_loss = []                                                                                                                                                                                  
log_validation_loss = []                                                                                                                                                                                                                                                                                                                                                                                       
log_training_acc = []
log_validation_acc = []


In [325]:
def get_loss(y,a) :
    #print(y,a)
    ret = -(torch.div(y,a) - torch.div(1-y,1-a))
    
    return ret


class ML :
    def __init__(self,t_data,t_yh,v_data,v_yh,L1_size,L2_size,L3_size,activation_type,lr) :
        self.t_data = t_data
        self.t_data_size = self.t_data.shape[0]
        self.feature_size = self.t_data.shape[1]
        self.t_yh = t_yh
        self.v_data = v_data
        self.v_yh = v_yh
        self.l1_size = L1_size
        self.l2_size = L2_size
        self.l3_size = L3_size
        self.batch_size = 256
        self.t_batch_idx = 0
        self.epoch = 0
        self.lr = lr
        self.act_type = activation_type
        self.init_weights()
        
        print("ML Object initialized")
        self.iter = 0
        

    def get_training_batch(self) : #(batchs, 10000)
        self.idx_1 = self.t_batch_idx * self.batch_size
        self.idx_2 = self.idx_1 + self.batch_size
        self.idx_3 = 0
        if self.idx_2 > self.t_data_size :
            #print(self.idx_1,self.idx_2,self.idx_3)
            self.idx_3 = self.idx_2 - self.t_data_size
            self.idx_2 = self.t_data_size
            
            tmp_tensor = self.t_data[self.idx_1:self.idx_2].clone().detach()
            tmp_tensor_2 = self.t_data[0:self.idx_3].clone().detach()
            
            
            ret = torch.cat((tmp_tensor,tmp_tensor_2),0)
            
            if (ret.shape[0] != self.batch_size) :
                print("ERROR at get batch, %s"%(ret.shape[1]))
            self.t_batch_idx = 0
            self.epoch += 1
        else :
            self.t_batch_idx += 1
            ret = self.t_data[self.idx_1:self.idx_2].clone().detach()

        
        return ret
    def get_t_yh_batch(self) :
        if self.idx_3 == 0 :
            ret = self.t_yh[self.idx_1:self.idx_2].clone().detach()
        else :
            tmp_tensor = self.t_yh[self.idx_1:self.idx_2].clone().detach()
            tmp_tensor_2 = self.t_yh[0:self.idx_3].clone().detach()
            
            ret = torch.cat((tmp_tensor,tmp_tensor_2),0)
        return ret

    
    def init_weights(self) :
        self.w_1 = torch.FloatTensor(self.l1_size,self.feature_size).uniform_(-1,1)
        self.b_1 = torch.FloatTensor(1,self.l1_size).uniform_(-1,1)
        
        #torch.FloatTensor(a, b).uniform_(r1, r2)
        
        self.w_2 = torch.FloatTensor(self.l2_size,self.l1_size).uniform_(-1,1)
        self.b_2 = torch.FloatTensor(1,self.l2_size).uniform_(-1,1)

        self.w_3 = torch.FloatTensor(self.l3_size,self.l2_size).uniform_(-1,1)
        self.b_3 = torch.FloatTensor(1,self.l3_size).uniform_(-1,1)

        

        
    def training(self) :
    
        while(True) :
            self.t_data_batch = self.get_training_batch()
            self.z_1 = torch.matmul(self.t_data_batch,self.w_1.T) + self.b_1
            
            self.a_1 = get_activation(self.z_1,self.act_type)
            
            
            self.z_2 = torch.matmul(self.a_1,self.w_2.T) + self.b_2
            self.a_2 = get_activation(self.z_2,self.act_type) # 1027 x 50

            self.z_3 = torch.matmul(self.a_2,self.w_3.T) + self.b_3
            self.a_3 = get_activation(self.z_3,self.act_type) # 1027 x 1

            self.t_yh_batch = self.get_t_yh_batch()

            acc = self.get_acc(self.a_3,self.t_yh_batch)
            

            print("iter : %s, epoch : %s, loss : %s, acc : %s"%(self.iter,self.epoch,0,acc))

            self.update_weights(self.t_yh_batch,self.a_3)
            print(np.array(get_loss(self.t_yh_batch, self.a_3)).mean())
            self.validation()
            #break
            if self.epoch > 1000000 :
                break
    
    def validation(self) :
        self.v_data_batch = self.v_data
        self.v_yh_batch = self.v_yh
        
        self.z_1 = torch.matmul(self.v_data_batch,self.w_1.T) + self.b_1
        self.a_1 = get_activation(self.z_1,self.act_type) # Activation 함수 적용
        self.z_2 = torch.matmul(self.a_1,self.w_2.T) + self.b_2
        self.a_2 = get_activation(self.z_2,self.act_type) # Activation 함수 적용
        self.z_3 = torch.matmul(self.a_2,self.w_3.T) + self.b_3
        self.a_3 = get_activation(self.z_3,self.act_type) # y^과 같음
        
        
        
        acc = self.get_acc(self.a_3,self.v_yh_batch)
        print("val acc : %s"%(acc))
    
        
    def update_weights(self,t_y,a_3) :
        #print("t_y:",t_y,"z_3",z_3)
        error_wb3 = -(torch.div(t_y,a_3+0.00000001) - torch.div(1-t_y,1-a_3+0.00000001))
        d_z_3 = error_wb3*get_derv_activation(self.z_3,self.act_type) #         
        d_w_3 = torch.matmul(d_z_3.T,self.a_2)
        d_b_3 = torch.sum(d_z_3, dim=0, keepdim=True) / self.a_2.shape[0] # mean도 됨
        
        #########
        
        error_wb2 = torch.matmul(d_z_3,self.w_3)
        
        d_z_2 = error_wb2*get_derv_activation(self.z_2,self.act_type)
        
        d_w_2 = torch.matmul(d_z_2.T,self.a_1)
        d_b_2 = torch.sum(d_z_2, dim=0,keepdims=True) / self.a_1.shape[0]
        
   
        error_wb1 = torch.matmul(d_z_2,self.w_2)
        d_z_1 = error_wb1*get_derv_activation(self.z_1,self.act_type)
        d_w_1 = torch.matmul(d_z_1.T,self.t_data_batch)
        d_b_1 = torch.sum(d_z_1, dim=0,keepdims=True) / self.t_data_batch.shape[0]
        
        self.w_3 += -self.lr*d_w_3
        self.b_3 += -self.lr*d_b_3
        
        self.w_2 += -self.lr*d_w_2
        self.b_2 += -self.lr*d_b_2
        self.w_1 += -self.lr*d_w_1
        self.b_1 += -self.lr*d_b_1
        #print(b_3,b_2,b_1)
        
    def get_acc(self,yhat,y) :
        count = 0
        
        for a,b in zip(yhat,y) :
            if a >= 0.5 :
                if b == 1 :
                    count+=1
            else :
                if b == 0:
                    count +=1
        
        return count / len(yhat)

 

In [326]:
t_data = torch.FloatTensor(training_vectorized)
t_yh = torch.FloatTensor(training_labels)
v_data = torch.FloatTensor(validation_vectorized)
v_yh = torch.FloatTensor(validation_labels)
print(t_data.shape,t_yh.shape,v_data.shape,v_yh.shape)

activation_type = 0 # 0 = sigmoid
learningRate = 0.01
machine = ML(t_data,t_yh,v_data,v_yh, 100,50,1,0,learningRate)

torch.Size([1027, 10000]) torch.Size([1027, 1]) torch.Size([256, 10000]) torch.Size([256, 1])
ML Object initialized


In [ ]:
machine.training()

iter : 0, epoch : 0, loss : 0, acc : 0.515625
-0.04597131
val acc : 0.5
iter : 0, epoch : 0, loss : 0, acc : 0.5234375
-0.072826006
val acc : 0.5
iter : 0, epoch : 0, loss : 0, acc : 0.45703125
0.1991306
val acc : 0.5
iter : 0, epoch : 0, loss : 0, acc : 0.5546875
-0.20813318
val acc : 0.5
iter : 0, epoch : 1, loss : 0, acc : 0.51171875
-0.018700808
val acc : 0.5
iter : 0, epoch : 1, loss : 0, acc : 0.515625
-0.032767937
val acc : 0.5
iter : 0, epoch : 1, loss : 0, acc : 0.5234375
-0.061269782
val acc : 0.5
iter : 0, epoch : 1, loss : 0, acc : 0.45703125
0.20947434
val acc : 0.5
iter : 0, epoch : 1, loss : 0, acc : 0.5546875
-0.19921444
val acc : 0.5
iter : 0, epoch : 2, loss : 0, acc : 0.51171875
-0.009905167
val acc : 0.5
iter : 0, epoch : 2, loss : 0, acc : 0.515625
-0.024904042
val acc : 0.5
iter : 0, epoch : 2, loss : 0, acc : 0.5234375
-0.054062627
val acc : 0.5
iter : 0, epoch : 2, loss : 0, acc : 0.45703125
0.21613052
val acc : 0.5
iter : 0, epoch : 2, loss : 0, acc : 0.5546875

0.008577719
val acc : 0.5
iter : 0, epoch : 23, loss : 0, acc : 0.515625
-0.008161068
val acc : 0.5
iter : 0, epoch : 23, loss : 0, acc : 0.5234375
-0.04250405
val acc : 0.5
iter : 0, epoch : 23, loss : 0, acc : 0.45703125
0.22234783
val acc : 0.5
iter : 0, epoch : 23, loss : 0, acc : 0.5546875
-0.18243314
val acc : 0.5
iter : 0, epoch : 24, loss : 0, acc : 0.51171875
0.0057429597
val acc : 0.5
iter : 0, epoch : 24, loss : 0, acc : 0.515625
-0.014353156
val acc : 0.5
iter : 0, epoch : 24, loss : 0, acc : 0.5234375
-0.04388976
val acc : 0.5
iter : 0, epoch : 24, loss : 0, acc : 0.45703125
0.22316793
val acc : 0.5
iter : 0, epoch : 24, loss : 0, acc : 0.5546875
-0.18777525
val acc : 0.5
iter : 0, epoch : 25, loss : 0, acc : 0.51171875
0.0049304813
val acc : 0.5
iter : 0, epoch : 25, loss : 0, acc : 0.515625
-0.009311035
val acc : 0.5
iter : 0, epoch : 25, loss : 0, acc : 0.5234375
-0.04102783
val acc : 0.5
iter : 0, epoch : 25, loss : 0, acc : 0.45703125
0.22840224
val acc : 0.5
iter : 0

-0.0027493834
val acc : 0.5
iter : 0, epoch : 45, loss : 0, acc : 0.5234375
-0.03783908
val acc : 0.5
iter : 0, epoch : 45, loss : 0, acc : 0.45703125
0.2375165
val acc : 0.5
iter : 0, epoch : 45, loss : 0, acc : 0.5546875
-0.17429106
val acc : 0.5
iter : 0, epoch : 46, loss : 0, acc : 0.51171875
0.011013314
val acc : 0.5
iter : 0, epoch : 46, loss : 0, acc : 0.515625
9.0524554e-05
val acc : 0.5
iter : 0, epoch : 46, loss : 0, acc : 0.5234375
-0.033035442
val acc : 0.5
iter : 0, epoch : 46, loss : 0, acc : 0.45703125
0.2284421
val acc : 0.5
iter : 0, epoch : 46, loss : 0, acc : 0.5546875
-0.1807553
val acc : 0.5
iter : 0, epoch : 47, loss : 0, acc : 0.51171875
0.0027476698
val acc : 0.5
iter : 0, epoch : 47, loss : 0, acc : 0.515625
-0.002891302
val acc : 0.5
iter : 0, epoch : 47, loss : 0, acc : 0.5234375
-0.034065314
val acc : 0.5
iter : 0, epoch : 47, loss : 0, acc : 0.45703125
0.23460662
val acc : 0.5
iter : 0, epoch : 47, loss : 0, acc : 0.5546875
-0.18003151
val acc : 0.5
iter : 

iter : 0, epoch : 68, loss : 0, acc : 0.51171875
-0.005179994
val acc : 0.5
iter : 0, epoch : 68, loss : 0, acc : 0.515625
-0.020146355
val acc : 0.5
iter : 0, epoch : 68, loss : 0, acc : 0.5234375
-0.049112976
val acc : 0.5
iter : 0, epoch : 68, loss : 0, acc : 0.45703125
0.22206059
val acc : 0.5
iter : 0, epoch : 68, loss : 0, acc : 0.5546875
-0.19292079
val acc : 0.5
iter : 0, epoch : 69, loss : 0, acc : 0.51171875
-0.00019805878
val acc : 0.5
iter : 0, epoch : 69, loss : 0, acc : 0.515625
-0.01585047
val acc : 0.5
iter : 0, epoch : 69, loss : 0, acc : 0.5234375
-0.045078557
val acc : 0.5
iter : 0, epoch : 69, loss : 0, acc : 0.45703125
0.22546156
val acc : 0.5
iter : 0, epoch : 69, loss : 0, acc : 0.5546875
-0.18978092
val acc : 0.5
iter : 0, epoch : 70, loss : 0, acc : 0.51171875
0.0029811785
val acc : 0.5
iter : 0, epoch : 70, loss : 0, acc : 0.515625
-0.00907889
val acc : 0.5
iter : 0, epoch : 70, loss : 0, acc : 0.5234375
-0.043129783
val acc : 0.5
iter : 0, epoch : 70, loss : 

val acc : 0.5
iter : 0, epoch : 91, loss : 0, acc : 0.515625
-0.008606479
val acc : 0.5
iter : 0, epoch : 91, loss : 0, acc : 0.5234375
-0.036411963
val acc : 0.5
iter : 0, epoch : 91, loss : 0, acc : 0.45703125
0.23441952
val acc : 0.5
iter : 0, epoch : 91, loss : 0, acc : 0.5546875
-0.18151043
val acc : 0.5
iter : 0, epoch : 92, loss : 0, acc : 0.51171875
0.012011737
val acc : 0.5
iter : 0, epoch : 92, loss : 0, acc : 0.515625
-0.0037149936
val acc : 0.5
iter : 0, epoch : 92, loss : 0, acc : 0.5234375
-0.034657147
val acc : 0.5
iter : 0, epoch : 92, loss : 0, acc : 0.45703125
0.23521268
val acc : 0.5
iter : 0, epoch : 92, loss : 0, acc : 0.5546875
-0.18361212
val acc : 0.5
iter : 0, epoch : 93, loss : 0, acc : 0.51171875
0.009840563
val acc : 0.5
iter : 0, epoch : 93, loss : 0, acc : 0.515625
-0.009339571
val acc : 0.5
iter : 0, epoch : 93, loss : 0, acc : 0.5234375
-0.036615837
val acc : 0.5
iter : 0, epoch : 93, loss : 0, acc : 0.45703125
0.23352899
val acc : 0.5
iter : 0, epoch : 

...

val acc : 0.5
iter : 0, epoch : 2467, loss : 0, acc : 0.45703125
0.22625111
val acc : 0.5
iter : 0, epoch : 2467, loss : 0, acc : 0.5546875
-0.17812496
val acc : 0.5
iter : 0, epoch : 2468, loss : 0, acc : 0.51171875
0.004390955
val acc : 0.5
iter : 0, epoch : 2468, loss : 0, acc : 0.515625
-0.011505783
val acc : 0.5
iter : 0, epoch : 2468, loss : 0, acc : 0.5234375
-0.042090375
val acc : 0.5
iter : 0, epoch : 2468, loss : 0, acc : 0.45703125
0.22624016
val acc : 0.5
iter : 0, epoch : 2468, loss : 0, acc : 0.5546875
-0.1781004
val acc : 0.5
iter : 0, epoch : 2469, loss : 0, acc : 0.51171875
0.0043868273
val acc : 0.5
iter : 0, epoch : 2469, loss : 0, acc : 0.515625
-0.011508852
val acc : 0.5
iter : 0, epoch : 2469, loss : 0, acc : 0.5234375
-0.04209414
val acc : 0.5
iter : 0, epoch : 2469, loss : 0, acc : 0.45703125
0.22623019
val acc : 0.5
iter : 0, epoch : 2469, loss : 0, acc : 0.5546875
-0.17807375
val acc : 0.5
iter : 0, epoch : 2470, loss : 0, acc : 0.51171875
0.004385233
val acc 

0.004359022
val acc : 0.5
iter : 0, epoch : 2490, loss : 0, acc : 0.515625
-0.01152809
val acc : 0.5
iter : 0, epoch : 2490, loss : 0, acc : 0.5234375
-0.042132825
val acc : 0.5
iter : 0, epoch : 2490, loss : 0, acc : 0.45703125
0.2261215
val acc : 0.5
iter : 0, epoch : 2490, loss : 0, acc : 0.5546875
-0.17779692
val acc : 0.5
iter : 0, epoch : 2491, loss : 0, acc : 0.51171875
0.004343763
val acc : 0.5
iter : 0, epoch : 2491, loss : 0, acc : 0.515625
-0.011541888
val acc : 0.5
iter : 0, epoch : 2491, loss : 0, acc : 0.5234375
-0.042147536
val acc : 0.5
iter : 0, epoch : 2491, loss : 0, acc : 0.45703125
0.22610128
val acc : 0.5
iter : 0, epoch : 2491, loss : 0, acc : 0.5546875
-0.1777822
val acc : 0.5
iter : 0, epoch : 2492, loss : 0, acc : 0.51171875
0.0043309033
val acc : 0.5
iter : 0, epoch : 2492, loss : 0, acc : 0.515625
-0.011553183
val acc : 0.5
iter : 0, epoch : 2492, loss : 0, acc : 0.5234375
-0.04215988
val acc : 0.5
iter : 0, epoch : 2492, loss : 0, acc : 0.45703125
0.2260834

-0.011647433
val acc : 0.5
iter : 0, epoch : 2512, loss : 0, acc : 0.5234375
-0.042280927
val acc : 0.5
iter : 0, epoch : 2512, loss : 0, acc : 0.45703125
0.22584882
val acc : 0.5
iter : 0, epoch : 2512, loss : 0, acc : 0.5546875
-0.17735304
val acc : 0.5
iter : 0, epoch : 2513, loss : 0, acc : 0.51171875
0.0042144284
val acc : 0.5
iter : 0, epoch : 2513, loss : 0, acc : 0.515625
-0.011650592
val acc : 0.5
iter : 0, epoch : 2513, loss : 0, acc : 0.5234375
-0.042286076
val acc : 0.5
iter : 0, epoch : 2513, loss : 0, acc : 0.45703125
0.22583863
val acc : 0.5
iter : 0, epoch : 2513, loss : 0, acc : 0.5546875
-0.17733286
val acc : 0.5
iter : 0, epoch : 2514, loss : 0, acc : 0.51171875
0.0042102486
val acc : 0.5
iter : 0, epoch : 2514, loss : 0, acc : 0.515625
-0.011654228
val acc : 0.5
iter : 0, epoch : 2514, loss : 0, acc : 0.5234375
-0.042290863
val acc : 0.5
iter : 0, epoch : 2514, loss : 0, acc : 0.45703125
0.22582826
val acc : 0.5
iter : 0, epoch : 2514, loss : 0, acc : 0.5546875
-0.1

iter : 0, epoch : 2535, loss : 0, acc : 0.51171875
0.0041247755
val acc : 0.5
iter : 0, epoch : 2535, loss : 0, acc : 0.515625
-0.011723697
val acc : 0.5
iter : 0, epoch : 2535, loss : 0, acc : 0.5234375
-0.042387687
val acc : 0.5
iter : 0, epoch : 2535, loss : 0, acc : 0.45703125
0.22562265
val acc : 0.5
iter : 0, epoch : 2535, loss : 0, acc : 0.5546875
-0.17690748
val acc : 0.5
iter : 0, epoch : 2536, loss : 0, acc : 0.51171875
0.0041209534
val acc : 0.5
iter : 0, epoch : 2536, loss : 0, acc : 0.515625
-0.011727005
val acc : 0.5
iter : 0, epoch : 2536, loss : 0, acc : 0.5234375
-0.042392094
val acc : 0.5
iter : 0, epoch : 2536, loss : 0, acc : 0.45703125
0.22561313
val acc : 0.5
iter : 0, epoch : 2536, loss : 0, acc : 0.5546875
-0.17688915
val acc : 0.5
iter : 0, epoch : 2537, loss : 0, acc : 0.51171875
0.004117079
val acc : 0.5
iter : 0, epoch : 2537, loss : 0, acc : 0.515625
-0.011730075
val acc : 0.5
iter : 0, epoch : 2537, loss : 0, acc : 0.5234375
-0.042396493
val acc : 0.5
iter

iter : 0, epoch : 2557, loss : 0, acc : 0.5234375
-0.04248062
val acc : 0.5
iter : 0, epoch : 2557, loss : 0, acc : 0.45703125
0.22542578
val acc : 0.5
iter : 0, epoch : 2557, loss : 0, acc : 0.5546875
-0.17652395
val acc : 0.5
iter : 0, epoch : 2558, loss : 0, acc : 0.51171875
0.0040404946
val acc : 0.5
iter : 0, epoch : 2558, loss : 0, acc : 0.515625
-0.011793077
val acc : 0.5
iter : 0, epoch : 2558, loss : 0, acc : 0.5234375
-0.042484507
val acc : 0.5
iter : 0, epoch : 2558, loss : 0, acc : 0.45703125
0.22541738
